For now, starter code that compares lines from the two files and prints out the differences.

In [1]:
import json, csv
import pandas as pd
from constants import *
import numpy as np
from scipy.stats import poisson

def poisson_over_likelyhood(mean, target):
    return 1 - poisson.cdf(target, mean)



C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The following code will web scrape to generate the player data, feel free to ingore

In [13]:
# import fbref_scrape
# import pp_scrape

In [2]:
players = pd.read_csv("Premier_League_Player_Stats.csv")
PP_lines = pd.read_csv("pp_lines.csv")

merged = pd.merge(players, PP_lines, on="name")
merged

,name,team,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,...,Interceptions,Passes Attempted,Shots Assisted,Shots On Target,Goalie Saves,Clearances,Fouls Drawn,Goal + Assist,Fouls,Goals
0,Conor Gallagher,Chelsea,29,2421,NaN,15.0,70.0,45.0,42.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,Axel Disasi,Chelsea,28,2480,NaN,97.0,31.0,3.0,17.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Moises Caicedo,Chelsea,27,2173,NaN,32.0,62.0,15.0,36.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Enzo Fernandez,Chelsea,27,2143,NaN,28.0,48.0,32.0,31.0,44.0,...,NaN,79.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nicolas Jackson,Chelsea,27,2084,NaN,15.0,14.0,26.0,23.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Thiago Silva,Chelsea,24,2073,NaN,117.0,26.0,6.0,8.0,10.0,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
6,Cole Palmer,Chelsea,26,1993,NaN,11.0,21.0,52.0,20.0,69.0,...,NaN,47.5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.5
7,Raheem Sterling,Chelsea,27,1851,NaN,8.0,20.0,26.0,37.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Malo Gusto,Chelsea,22,1528,NaN,36.0,50.0,29.0,27.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Mykhailo Mudryk,Chelsea,24,1080,NaN,6.0,20.0,20.0,20.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
merged = merged.loc[merged['minutes'] > 200].loc[merged['games'] > 5]

merged.loc[merged['Goalie Saves'] > 0].head()

,name,team,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,...,Interceptions,Passes Attempted,Shots Assisted,Shots On Target,Goalie Saves,Clearances,Fouls Drawn,Goal + Assist,Fouls,Goals


We need a new dataframe such that each row repesents a singular betting line that has a per-90 line, a per-game line, and a prizepicks line

In [3]:
results = []

for index, row in merged.iterrows():
    # Access the values of each column in the current row
    name = row['name']

    for pp_stat_type in pp_to_fbref_stats.keys():
        # failsafe 
        if pp_stat_type not in row:
            continue

        if row[pp_stat_type] <= 0:
            continue

        # Create a dictionary to store the result
        result_dict = {
            'name': name,
            'stat': pp_stat_type,
            'line': row[pp_stat_type]
        }

        # Get the FBref stat type that corresponds to the current PP stat type
        fbref_stat_type = pp_to_fbref_stats[pp_stat_type]

        # Get the value of the FBref stat type for the current player
        fbref_stat_value = row[fbref_stat_type]

        # Add the FBref stat value to the result dictionary
        result_dict['per90'] = fbref_stat_value * 90 / row['minutes']
        result_dict['perGame'] = fbref_stat_value / row['games']


        # Append the dictionary to the results list
        results.append(result_dict)

# Convert the results list into a dataframe
df = pd.DataFrame(results).dropna()

df


,name,stat,line,per90,perGame
5,Conor Gallagher,Shots,1.5,1.486989,1.379310
9,Conor Gallagher,Fouls,2.0,2.379182,2.206897
16,Axel Disasi,Shots,0.5,0.544355,0.535714
24,Moises Caicedo,Tackles,2.0,2.567879,2.296296
38,Enzo Fernandez,Shots,2.0,1.847877,1.629630
40,Enzo Fernandez,Passes Attempted,79.5,75.804946,66.851852
49,Nicolas Jackson,Shots,2.5,2.461612,2.111111
56,Thiago Silva,Clearances,3.5,5.079595,4.875000
69,Cole Palmer,Shots Assisted,2.0,2.348219,2.000000
71,Cole Palmer,Shots,3.5,3.115906,2.653846


Now that we have that sorted, we can analyze the chart for interesting discrepencies in data

In [17]:
df = df.assign(quo = df['line'] / df['per90']).assign(diff = df['line'] - df['per90']).assign(over_likelyhood = poisson_over_likelyhood(df['per90'], df['line']))

df = df.sort_values(by='over_likelyhood', ascending=False)

# show the full thing
pd.set_option('display.max_rows', None)
df

,name,stat,line,per90,perGame,quo,diff,over_likelyhood
5,Ederson,Shots,1.5,0.0,0.0,inf,1.5,0.0
